<a href="https://colab.research.google.com/github/d-atallah/implicit_gender_bias/blob/main/word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [62]:
import json
import os

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from joblib import Parallel, delayed
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load Files

These files contain a sample of **social media posts** from the paper *RtGender: A Corpus for Studying Differential Responses to Gender* by Rob Voigt, David Jurgens, Vinodkumar Prabhakaran, Dan Jurafsky and Yulia Tsvetkov. Documentation is available [here](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fnlp.stanford.edu%2Frobvoigt%2Frtgender%2F). The sample includes an equal number of posts from the five data sources balanced on the gender of the original poster. Replacement was used to ensure less robust sources are adequately represented.

In [8]:
filepath = '/content/drive/MyDrive/SIADS 696: Milestone II/Project/Data/RtGender/sample'

In [9]:
filepath_train = os.path.join(filepath, 'train_five_million.csv')
filepath_validate = os.path.join(filepath, 'validate_five_million.csv')
filepath_test = os.path.join(filepath, 'test_five_million.csv')

In [11]:
dataframe_train = pd.read_csv(filepath_train)

In [42]:
dataframe_train.columns = ['source', 'source_id', 'op_gender', 'response_text', 'stratify']

In [43]:
dataframe_train.sample(5).sort_values('source_id')

,source,source_id,op_gender,response_text,stratify
332137,FBW,FBW2736851,M,Straight killin it lol,FBWM
916644,FIT,FIT144153,M,Anytime! Thanks for the follow.,FITM
808974,FIT,FIT29483,M,Thank you! Still waiting for that house call,FITM
1177013,TED,TED118488,M,Wonderful talk. As begin to peel off the secr...,TEDM
1439973,TED,TED51395,M,Thank you for taking on this project for the g...,TEDM


In [51]:
dataframe_train = dataframe_train.dropna()

This file contains the **stop words** available in the Natural Language Toolkit. Gendered pronouns have been removed.

In [14]:
filepath_stopwords = '/content/drive/MyDrive/SIADS 696: Milestone II/Project/Data/RtGender/stop_words.txt'

In [23]:
with open(filepath_stopwords, 'r') as file:
    stopwords = json.load(file)['stop_words']

This file contains **nouns** from the HolisticBias dataset, a project of the Responsible Natural Language Processing team at Facebook Research. The dataset is described in the paper *I'm sorry to hear that: Finding New Biases in Language Models with a Holistic Descriptor Dataset* by Eric Michael Smith, Melissa Hall, Melanie Kambadur, Eleonora Presani, and Adina Williams. Documentation is available [here](https://github.com/facebookresearch/ResponsibleNLP/tree/main/holistic_bias/dataset/v1.1).

In [15]:
filepath_nouns = '/content/drive/MyDrive/SIADS 696: Milestone II/Project/Data/RtGender/gendered_nouns.txt'
filepath_pronouns = '/content/drive/MyDrive/SIADS 696: Milestone II/Project/Data/RtGender/gendered_pronouns.txt'

In [29]:
with open (filepath_nouns, 'r') as file:
    nouns = json.load(file)

male_nouns = [item for sublist in nouns['male'] for item in sublist]
female_nouns = [item for sublist in nouns['female'] for item in sublist]

This file contains **pronouns** from Grammarly as described in the article *A Guide to Personal Pronouns and How They've Evolved*. The article includes additional neopronouns, pronouns that “refer to people entirely without reference to gender” (Grammarly, 2021). Documentation is available [here](https://www.grammarly.com/blog/gender-pronouns/).

In [30]:
with open(filepath_pronouns, 'r') as file:
    pronouns = json.load(file)

male_pronouns = pronouns['male']
female_pronouns = pronouns['female']

# Tokenize Text

In [34]:
class LemmaTokenizer:
    """
    A tokenizer class that optionally applies NLTK's WordNetLemmatizer to both tokens and stop words,
    and removes stop words based on a custom JSON file. The class can be configured to perform
    lemmatization, stop word removal, both, or neither, ensuring consistency between token and stop word
    processing. Based on code developed by Daniel Atallah.
    """

    def __init__(self, use_lemmatization=False, remove_stopwords=False, stopwords_file=None):
        """
        Initializes the LemmaTokenizer instance with options for lemmatization and stop word removal,
        and loads (and optionally lemmatizes) stop words from a specified JSON file if stop word removal
        is enabled.
        """
        self.use_lemmatization = use_lemmatization
        self.remove_stopwords = remove_stopwords and stopwords_file is not None
        self.lemmatizer = WordNetLemmatizer() if use_lemmatization else None
        self.tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
        self.stop_words = self._load_stopwords(stopwords_file) if self.remove_stopwords else set()

    def _load_stopwords(self, stopwords_file):
        """
        Loads and optionally lemmatizes stop words from a JSON file.
        """
        with open(stopwords_file, 'r') as file:
            stopwords = set(json.load(file))
        if self.use_lemmatization:
            return {self.lemmatizer.lemmatize(word) for word in stopwords}
        return stopwords

    def __call__(self, text):
        """
        Tokenizes and optionally lemmatizes and removes stop words from the input text.
        """
        tokens = self.tokenizer.tokenize(text)
        if self.use_lemmatization:
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        if self.remove_stopwords:
            tokens = [token for token in tokens if token.lower() not in self.stop_words]
        return tokens

In [35]:
tokenizer = LemmaTokenizer()

In [76]:
male_tokens = tokenizer(male_pronouns)
female_tokens = tokenizer(female_pronouns)

TypeError: expected string or buffer

In [57]:
dataframe_train['tokens'] = Parallel(n_jobs=-1)(delayed(tokenizer)(text) for text in dataframe_train['response_text'])

In [59]:
dataframe_train.head()

,source,source_id,op_gender,response_text,stratify,tokens
0,TED,TED5828,W,Beautiful... If only more people could see thi...,TEDW,"[beautiful, ..., if, only, more, people, could..."
1,RED,RED906982,M,Idk man. Cubs striking has looked NASTY lately...,REDM,"[idk, man, ., cubs, striking, has, looked, nas..."
2,FBW,FBW3327456,W,Having a hard time right now!! Im right there😥,FBWW,"[having, a, hard, time, right, now, !, !, im, ..."
3,FIT,FIT189959,W,Welcome! Fitocracy is a great place to track y...,FITW,"[welcome, !, fitocracy, is, a, great, place, t..."
4,RED,RED650418,W,Ich hab Sims4 deinstalliert nachdem ich eine H...,REDW,"[ich, hab, sims, 4, deinstalliert, nachdem, ic..."


# Train Model

In [61]:
model = Word2Vec(dataframe_train['tokens'], vector_size=100, window=5, min_count=5, sg=1, hs=0, negative=5, sample = 1e-3, workers = 4, epochs = 10)

In [66]:
model.wv.save('/content/drive/MyDrive/SIADS 696: Milestone II/Project/Models/initial.wordvectors')

# Calculate Bias

Garg et al. (2018) use a different approach to assess the similarity between a set of neutral words and two groups, first subtracting the distance between each group and a neutral word, then summing the results across words. This approach gives equal weight to each word, unlike the approach below. Documentation is available [here](https://pubmed.ncbi.nlm.nih.gov/29615513/).

In [74]:
import numpy as np
import pandas as pd

def add_bias(dataframe, token_column, male_tokens, female_tokens, model):
    """
    Calculate bias scores for text data in a DataFrame based on the difference in distances
    from male and female token embeddings.

    Parameters:
    - dataframe (pd.DataFrame): DataFrame containing the text data.
    - token_column (str): Column name containing the lists of tokens.
    - male_tokens (list of str): List of tokens associated with male attributes.
    - female_tokens (list of str): List of tokens associated with female attributes.
    - model: Model with a `get_mean_vector` method to compute embeddings.

    Returns:
    - pd.DataFrame: DataFrame with an additional 'bias' column.
    """
    # Compute embeddings and bias scores directly without intermediate columns
    male_vector = model.get_mean_vector(male_tokens)
    female_vector = model.get_mean_vector(female_tokens)

    def calculate_bias(tokens):
        # Ensure the tokens are passed correctly to the model's method
        embedding = model.get_mean_vector(tokens)
        bias = np.linalg.norm(male_vector - embedding) - np.linalg.norm(female_vector - embedding)
        return bias

    # Apply the combined operation, ensuring tokens are passed correctly to calculate_bias
    dataframe['bias'] = dataframe[token_column].apply(calculate_bias)

    return dataframe

In [75]:
add_bias(dataframe_train, 'response_text', male_pronouns, female_pronouns, model.wv)

,source,source_id,op_gender,response_text,stratify,tokens,bias
0,TED,TED5828,W,Beautiful... If only more people could see thi...,TEDW,"[beautiful, ..., if, only, more, people, could...",-0.054628
1,RED,RED906982,M,Idk man. Cubs striking has looked NASTY lately...,REDM,"[idk, man, ., cubs, striking, has, looked, nas...",-0.053548
2,FBW,FBW3327456,W,Having a hard time right now!! Im right there😥,FBWW,"[having, a, hard, time, right, now, !, !, im, ...",-0.057289
3,FIT,FIT189959,W,Welcome! Fitocracy is a great place to track y...,FITW,"[welcome, !, fitocracy, is, a, great, place, t...",-0.052358
4,RED,RED650418,W,Ich hab Sims4 deinstalliert nachdem ich eine H...,REDW,"[ich, hab, sims, 4, deinstalliert, nachdem, ic...",-0.053050
...,...,...,...,...,...,...,...
3499995,RED,RED382010,M,Bloody sex is the best sex behind Chloroform sex.,REDM,"[bloody, sex, is, the, best, sex, behind, chlo...",-0.054982
3499996,FBW,FBW4664155,W,Adorable,FBWW,[adorable],-0.054897
3499997,RED,RED1203857,W,I have been to 2 gynecologists. I have no phy...,REDW,"[i, have, been, to, 2, gynecologists, ., i, ha...",-0.051896
3499998,FIT,FIT21012,M,hahaahaah,FITM,[hahaahaah],-0.045187


This function uses Euclidean distance instead of cosine similarity. The advantage of using cosine similarity is that the distance between vectors is normalized. However, because the number of male and female nouns in the HolisticBias dataset is similar, it is not necessary to use a normalized measure, particularly if computational efficiency is compromised. Garg et al. (2018) also use Euclidean distance.

# Reduce Dimensions

In [ ]:
def train_vectorizer(text_data, vectorizer=TfidfVectorizer, tokenizer=TweetTokenizer()):
    """
    Trains a vectorizer on the provided text data and returns the vectorizer instance,
    the document-term matrix, and the feature names.

    Parameters:
    - text_data: List of text documents to be vectorized.
    - vectorizer: Vectorizer class to be used for text vectorization. Defaults to CountVectorizer.
    - tokenizer: Tokenizer class to be used for tokenizing the text documents. Defaults to TweetTokenizer.

    Returns:
    - instance: The trained vectorizer instance.
    - matrix: The document-term matrix resulting from fitting the vectorizer on `text_data`.
    - features: An array of feature names generated by the vectorizer.
    """
    # Initialize the vectorizer with specified configurations
    instance = vectorizer(
        strip_accents=None,  # Do not strip accents
        lowercase=False,  # Do not convert characters to lowercase
        tokenizer=tokenizer.tokenize,  # Use the tokenize method of the tokenizer instance
        token_pattern=None,  # Since a tokenizer is provided, token_pattern is not used
        stop_words=list(stop_words),  # Do not remove stop words
        ngram_range=(1, 1),  # Consider only single words (1-grams)
        min_df=0.01,  # Minimum document frequency for filtering terms
        max_df=0.99,  # Maximum document frequency for filtering terms
        max_features=None  # No limit on the number of features
    )

    # Fit the vectorizer on the provided text data and transform the data into a matrix
    matrix = instance.fit_transform(text_data)

    # Retrieve the feature names generated by the vectorizer
    features = instance.get_feature_names_out()

    return instance, matrix, features

In [ ]:
def train_svd(matrix, n_components=2, random_state=42):
    """
    Trains a Truncated Singular Value Decomposition (SVD) model on the given matrix.

    Parameters:
    - matrix: The input matrix to decompose.
    - n_components: Number of components to keep.
    - random_state: Seed for the random number generator.

    Returns:
    - A tuple containing the trained SVD model, term-topic matrix, document-topic matrix,
      and array of singular values.
    """
    svd = TruncatedSVD(n_components=n_components, random_state=random_state)
    model = svd.fit(np.transpose(matrix))
    term_topic_matrix = svd.transform(np.transpose(matrix))
    document_topic_matrix = svd.components_
    singular_values = svd.singular_values_

    return model, term_topic_matrix, document_topic_matrix, singular_values

# Visualize Data

In [ ]:
def plot_hist(dataframe, gender_column='op_gender', bias_column='bias'):

    fig, ax = plt.subplots()

    ax.hist(dataframe[dataframe[gender_column] == 'M'][bias_column], bins=100, density=True, alpha=0.5, label='Original Poster Male')
    ax.hist(dataframe[dataframe[gender_column] == 'W'][bias_column], bins=100, density=True, alpha=0.5, label='Original Poster Female')

    ax.set_title('Response Bias')
    ax.set_xlabel('Calculated Bias')
    ax.set_ylabel('Density')
    ax.legend()

    fig.show()

In [ ]:
def plot_svd(document_topic_matrix, mask):
    """
    Plots the SVD (Singular Value Decomposition) results, separating points by gender based on a mask.

    Parameters:
    - document_topic_matrix: The document-topic matrix obtained from SVD.
    - mask: An array of gender labels ('M' for male, 'W' for female) for each document.

    Returns:
    - None
    """
    mask_male = np.where(mask == 'M', True, False)
    mask_female = np.where(mask == 'W', True, False)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True, tight_layout=True)

    axs[0].scatter(document_topic_matrix[0][mask_male],
                   document_topic_matrix[1][mask_male],
                   alpha=0.1, color='C0')
    axs[0].set_title('Original Poster Male')

    axs[1].scatter(document_topic_matrix[0][mask_female],
                   document_topic_matrix[1][mask_female],
                   alpha=0.1, color='C1')
    axs[1].set_title('Original Poster Female')

    for ax in axs:
        ax.set_xlabel('Principal Component 1')
        ax.set_ylabel('Principal Component 2')

    plt.show()

# References

"Please annotate the following code and convert it into PEP 8." OpenAI. (2023). ChatGPT (Jan 30 version) [Large language model]. https://chat.openai.com/chat

Garg, N., Schiebinger, L., Jurafsky, D., & Zou, J. (2018). Word embeddings quantify 100 years of gender and ethnic stereotypes. PNAS, 115(16). https://doi.org/10.1073/pnas.1720347115